# About: Shibboleth認証設定

---

構築したMoodle環境でShibboleth認証を利用できるようにします。

## 概要

構築済のMoodle環境にShibbolethコンテナを追加し、MoodleのShibboleth認証の機能を有効にします。

![構成](images/moodle-220-01.png)

このNotebookではShibboleth認証の設定を行うために以下の手順で設定を行います。

1. 既存のMoodle環境にShibbolethコンテナを追加する
2. 設定ファイルの編集と反映を行う
3. MoodleのShibboleth認証プラグインの設定を行う

ここでは，MoodleがShibboleth SPとなり，Moodle利用時のユーザ認証にShibbolethを利用するようにします。
Shibboleth SPの構築にはIdPのメタデータが必要になるので，以下のどちらかを行っている必要があります。

* 学術認証フェデレーションへ参加しIdPが登録されている[参加申請](https://www.gakunin.jp/join)
* IdPのメタデータをIdP管理者や学内のDSを経由して取得している（IdPが学認に参加せず、学内システムとして構築する場合）

> 必要となる準備事項は環境毎に異なります。

また，IdPにはShibboleth SPのメタデータを登録する必要があります。これには，SP自体を学認に参加させてIdP読み込んでもらうか，IdPに対してSPのメタデータを送付するなどの方法があります。詳しくは，学内のIdP管理ポリシーに従ってください。

## 準備

### UnitGroup名の指定

このNotebookの操作対象となるAnsibleのグループ名を設定します。

Moodle環境を構築した際に「010-パラメータの設定.ipynb」で指定したUnitGroup名と同じ値を次のセルで指定してください。

In [ ]:
# (例)
# ugroup_name = 'Moodle'

ugroup_name =

#### チェック

指定された `ugroup_name` の値が適切なものかチェックします。

`ugroup_name` に対応する `group_vars` ファイルが存在していることを確認します。

In [ ]:
from pathlib import Path
if not (Path('group_vars') / (ugroup_name + '.yml')).exists():
    raise RuntimeError(f"ERROR: not exists {ugroup_name + '.yml'}")

`ugroup_name`に対応する VCノードが実行中であり Ansible で操作できることを確認します。

In [ ]:
!ansible {ugroup_name} -m ping

Moodleコンテナが起動していることを確認します。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle docker-compose ps'

### VCCアクセストークンの入力

VCCにアクセスするためのトークンを入力します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

#### チェック

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from vcpsdk.vcpsdk import VcpSDK
vcp = VcpSDK(vcc_access_token)

### 準備

これまでに他のNotebookで設定したパラメータを読み込む処理などを行います。

group_varsファイルに保存されているパラメータを読み込みます。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)

VCCのVault サーバにアクセスする際に必要となるパラメータを環境変数に設定します。

In [ ]:
import os
os.environ['VAULT_ADDR'] = vcp.vcc_info()['vault_url']
os.environ['VAULT_TOKEN'] = vcc_access_token

## Shibbolethコンテナの追加

現在のMoodle環境に Shibboleth コンテナを追加します。

![Shibbolethコンテナの追加](images/moodle-220-02.png)

ここではコンテナ構成を記述している `docker-compose.yml` を編集し Shibboleth コンテナを追加します。Shibboleth関連の設定ファイルなどは環境に合わせた変更が必要となります。Shibbolethコンテナに設定ファイルなど環境毎に異なるファイルを持たせないようにするために、コンテナの外側に設定ファイルをコピーする対応を併せて行います。

この節で行う操作の手順を以下に示します。

1. docker-compose.ymlの更新
    * Shibbolethコンテナの追加
    * プロキシコンテナから公開ポート(443)の設定を削除
2. コンテナ構成変更の反映
    * プロキシコンテナの再作成
    * Shibbolethコンテナの起動
3. Shibbolethコンテナから設定ファイルをコンテナの外側にコピーする

### docker-compose.yml の更新

`docker-compose.yml` にShibbolethコンテナを追加します。あわせてプロキシコンテナから公開ポートの削除も行います。

次のセルを実行するとShibbolethコンテナを追加した `docker-compose.yml` をローカル環境に作成し更新前との差分を表示します。また最後に表示されたリンクから更新後の `docker-compose.yml` を編集することもできます。

In [ ]:
%run scripts/edit_conf.py
append_shibboleth_container(
    ugroup_name, gvars['moodle_url'])

更新した`docker-compose.yml`をVCノードに配置します。

In [ ]:
upload_docker_compose(ugroup_name)

### コンテナ構成変更の反映

変更した`docker-compose.yml`をMoodle環境に適用します。

まずShibbolethコンテナのイメージを取得します。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle/ docker-compose pull shibboleth'

`docker-compose.yml`の変更を適用します。あわせてプロキシコンテナから公開ポートを削除するので、いったんプロキシコンテナを削除しています。

In [ ]:
!ansible {ugroup_name} -m shell -a 'chdir=/opt/moodle/ \
    docker-compose rm -s -f proxy && \
    docker-compose up -d'

コンテナが追加されたことを確認します。`shibboleth`コンテナが追加されたことと、すべてのコンテナの状態が `Up`となっていることを確認してください。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle/ \
    docker-compose ps'

### Shibbolethコンテナから設定ファイルをコンテナの外側にコピーする

Shibbolethコンテナでは Apache HTTP serverと shibd が実行されています。それらに関する設定ファイルをコンテナの外側にコピーします。

In [ ]:
!ansible {ugroup_name} -m shell -a 'chdir=/opt/moodle \
    mkdir -p shibboleth/conf/httpd && \
    docker cp shibboleth:/etc/shibboleth shibboleth/conf && \
    docker cp shibboleth:/etc/httpd/conf shibboleth/conf/httpd && \
    docker cp shibboleth:/etc/httpd/conf.d shibboleth/conf/httpd && \
    docker cp shibboleth:/etc/httpd/conf.modules.d shibboleth/conf/httpd'

コピーしたファイルを確認します。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle tree shibboleth/conf'

### Shibbolethコンテナに volume を追加する

コンテナ外にコピーした設定ファイルをShibbolethコンテナから参照できるようにするために`docker-compose.yml`にbind mountの設定を追加します。

次のセルを実行するとShibbolethコンテナにvolumeを追加した `docker-compose.yml` をローカル環境に作成し更新前との差分を表示します。また最後に表示されたリンクから更新後の `docker-compose.yml` を編集することもできます。

In [ ]:
%run scripts/edit_conf.py
append_shibboleth_container(
    ugroup_name, gvars['moodle_url'],
    volumes=[
        '/opt/moodle/shibboleth/conf/shibboleth:/etc/shibboleth',
        '/opt/moodle/shibboleth/conf/httpd/conf:/etc/httpd/conf',
        '/opt/moodle/shibboleth/conf/httpd/conf.d:/etc/httpd/conf.d',
        '/opt/moodle/shibboleth/conf/httpd/conf.modules.d:/etc/httpd/conf.modules.d',
        '/opt/moodle/shibboleth/conf/httpd/cert:/etc/httpd/cert',
    ],
)

更新した`docker-compose.yml`をVCノードに配置して、コンテナ構成の変更を反映します。

In [ ]:
upload_docker_compose(ugroup_name, apply=True)

コンテナ構成変更後にすべてのコンテナの状態が `Up`となっていることを確認します。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle/ \
    docker-compose ps'

## 設定ファイルの編集と反映を行う

Shibbolethに関する設定ファイルの編集とその反映をおこないます。

![設定ファイルの編集](images/moodle-220-03.png)

ここでは、前節でコピーしたShibbolethに関する設定ファイルの編集を行います。編集対象となるファイルのShibbolethコンテナにおけるパスとVCノード（コンテナ外）におけるパスの対応関係を以下の表に示します。

<table>
  <tr>
    <th style="text-align:left;">コンテナ内のパス</th>
    <th style="text-align:left;">VCノードのパス</th>
  </tr>
  <tr>
    <td style="text-align:left;">/etc/shibboleth/</td>
    <td style="text-align:left;">/opt/moodle/shibboleth/conf/shibboleth/</td>
  </tr>
  <tr>
    <td style="text-align:left;">/etc/httpd/conf/</td>
    <td style="text-align:left;">/opt/moodle/shibboleth/conf/httpd/conf/</td>
  </tr>
  <tr>
    <td style="text-align:left;">/etc/httpd/conf.d/</td>
    <td style="text-align:left;">/opt/moodle/shibboleth/conf/httpd/conf.d/</td>
  </tr>
  <tr>
    <td style="text-align:left;">/etc/httpd/conf.modules.d/</td>
    <td style="text-align:left;">/opt/moodle/shibboleth/conf/httpd/conf.modules.d/</td>
  </tr>
  <tr>
    <td style="text-align:left;">/etc/httpd/cert/</td>
    <td style="text-align:left;">/opt/moodle/shibboleth/conf/httpd/cert/</td>
  </tr>
</table>

具体的な例として `shibboleth2.xml`, `shib.conf`, `httpd.conf`, `ssl.conf` の対応関係を以下の表に示します。

<table>
  <tr>
    <th style="text-align:left;">コンテナ内のパス</th>
    <th style="text-align:left;">VCノードのパス</th>
  </tr>
  <tr>
    <td style="text-align:left;">/etc/shibboleth/shibboleth2.xml</td>
    <td style="text-align:left;">/opt/moodle/shibboleth/conf/shibboleth/shibboleth2.xml</td>
  <tr>
    <td style="text-align:left;">/etc/httpd/conf.d/shib.conf</td>
    <td style="text-align:left;">/opt/moodle/shibboleth/conf/httpd/conf.d/shib.conf</td>
  <tr>
    <td style="text-align:left;">/etc/httpd/conf/httpd.conf</td>
    <td style="text-align:left;">/opt/moodle/shibboleth/conf/httpd/conf/httpd.conf</td>
  <tr>
    <td style="text-align:left;">/etc/httpd/conf.d/ssl.conf</td>
    <td style="text-align:left;">/opt/moodle/shibboleth/conf/httpd/conf.d/ssl.conf</td>
</table>

Shibbolethに関する設定は環境や関連するサーバ構成などによって設定内容がおおきく異なるためこのNotebookでは詳細な説明を行いません。以下の資料などを参照して設定を行ってください。

* [SPセッティング](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=12158187)
* [学内システムとして構築する場合の設定](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=12158282)

以下のような設定ファイルを編集、配置する必要があります。

* /etc/shibboleth/shibboleth2.xmlの編集
* サーバ証明書の配置
* IdPメタデータの配置(学内システムとして構築する場合)
* /etc/httpd/conf.d/ssl.confの編集
    - サーバ証明書の設定
    - ホスト名の設定

### 設定ファイルの編集

編集が必要となる設定ファイルは環境毎に異なるので、例としてここでは以下の操作を行う手順を示します。

* /etc/shibboleth/shibboleth2.xmlの編集
* サーバ証明書の配置
* IdPメタデータの配置(学内システムとして構築する場合)
* /etc/httpd/conf.d/ssl.confの編集

その他の設定ファイル（proxy.confやnative.logger, shibd.logger, attribute-map.xmlなど）を変更が必要な場合は以下のセルをコピーして同様の操作を行ってください。

#### shibboleth2.xml の編集

「030-設定ファイルの変更.ipynb」で行っているのと同様の手順で設定ファイルの編集を行います。
操作手順は以下のようになります。

1. VCノードに配置されている設定ファイルをNotebook環境に取得する
2. 取得したファイルのバックアップを作成する
3. Notebookの編集機能を利用して設定ファイルの変更をおこなう
4. 変更した設定ファイルをVCノードに配置する

> `shibboleth2.xml`の記述方法については学認の「[shibboleth2.xml ファイル](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=12158266)」、「[学内システムとして構築する場合の設定](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=12158282)」などを参照してください。

まず対象となるコンテナを指定します。ここではShibbolethコンテナを対象としているので `shibboleth` を指定します。

In [ ]:
target_container = 'shibboleth'

次に編集対象となる設定ファイルのパスを指定します。指定するのはコンテナ内のパスとなります。

In [ ]:
target_file = '/etc/shibboleth/shibboleth2.xml'

設定ファイル`shibboleth2.xml`をローカル環境に取得しJupyter Notebookの編集機能を用いて編集を行います。次のセルを実行するとリンクが表示されるのでクリックして編集を行ってください。

> ファイルの編集後は**必ず**、メニューの[File]-[Save]を選択してファイルの保存を行ってください。

In [ ]:
%run scripts/edit_conf.py
fetch_conf(ugroup_name, target_container, target_file)

ローカル環境に取得したファイルは、以下のパスに格納されています。

`./edit/{ugroup_name}/{YYYYMMDDHHmmssffffff}/shibboleth2.xml`

`{ugroup_name}` には UnitGroup名が、`{YYYYMMDDHHmmssfffff}` にはファイルを取得したタイムスタンプが入ります。

また、バックアップファイルは以下のパスに格納されます。

`./edit/{ugroup_name}/{YYYYMMDDHHmmssffffff}/shibboleth2.xml.orig`

次のセルを実行すると編集の前後での差分を確認することができます。

In [ ]:
show_local_conf_diff(ugroup_name, target_container, target_file)

設定ファイルの配置を行います。次のセルを実行すると以下の手順が実行されます。

1. 編集前と編集後の設定ファイルの差分を表示する
2. 編集した設定ファイル(`shibboleth2.xml`)をVCノードに配置する

In [ ]:
apply_conf(ugroup_name, target_container, target_file, restart=False)

#### サーバ証明書の配置

まず対象となるコンテナを指定します。ここではShibbolethコンテナを対象としているので `shibboleth` を指定します。

In [ ]:
target_container = 'shibboleth'

次に編集対象となる設定ファイルのパスを指定します。指定するのはコンテナ内のパスとなります。

In [ ]:
target_file = '/etc/httpd/cert/server.crt'

空のサーバ証明書`server.crt`ファイルをローカル環境に作成しJupyter Notebookの編集機能を用いて編集を行います。次のセルを実行するとリンクが表示されるのでクリックして編集を行ってください。

> ファイルの編集後は**必ず**、メニューの[File]-[Save]を選択してファイルの保存を行ってください。

In [ ]:
%run scripts/edit_conf.py
create_conf(ugroup_name, target_container, target_file)

作成したファイルは、以下のパスに格納されています。

`./edit/{ugroup_name}/{YYYYMMDDHHmmssffffff}/shibboleth2.xml`

`{ugroup_name}` には UnitGroup名が、`{YYYYMMDDHHmmssfffff}` にはファイルを取得したタイムスタンプが入ります。

設定ファイルの配置を行います。次のセルを実行すると以下の手順が実行されます。

1. 編集前と編集後の設定ファイルの差分を表示する
2. 編集した設定ファイルをVCノードに配置する

In [ ]:
%run scripts/edit_conf.py
apply_conf(ugroup_name, target_container, target_file, restart=False)

#### サーバ証明書の秘密鍵の配置

まず対象となるコンテナを指定します。ここではShibbolethコンテナを対象としているので `shibboleth` を指定します。

In [ ]:
target_container = 'shibboleth'

次に編集対象となる設定ファイルのパスを指定します。指定するのはコンテナ内のパスとなります。

In [ ]:
target_file = '/etc/httpd/cert/server.key'

空の秘密鍵ファイル`server.key`をローカル環境に作成しJupyter Notebookの編集機能を用いて編集を行います。次のセルを実行するとリンクが表示されるのでクリックして編集を行ってください。

> ファイルの編集後は**必ず**、メニューの[File]-[Save]を選択してファイルの保存を行ってください。

In [ ]:
%run scripts/edit_conf.py
create_conf(ugroup_name, target_container, target_file)

作成したファイルは、以下のパスに格納されています。

`./edit/{ugroup_name}/{YYYYMMDDHHmmssffffff}/shibboleth2.xml`

`{ugroup_name}` には UnitGroup名が、`{YYYYMMDDHHmmssfffff}` にはファイルを取得したタイムスタンプが入ります。

設定ファイルの配置を行います。次のセルを実行すると以下の手順が実行されます。

1. 編集前と編集後の設定ファイルの差分を表示する
2. 編集した設定ファイルをVCノードに配置する

In [ ]:
%run scripts/edit_conf.py
apply_conf(ugroup_name, target_container, target_file, restart=False)

#### IdPメタデータの配置

まず対象となるコンテナを指定します。ここではShibbolethコンテナを対象としているので `shibboleth` を指定します。

In [ ]:
target_container = 'shibboleth'

次に編集対象となる設定ファイルのパスを指定します。指定するのはコンテナ内のパスとなります。

In [ ]:
target_file = '/etc/shibboleth/idp-metadata.xml'

空のファイルをローカル環境に作成しJupyter Notebookの編集機能を用いて編集を行います。次のセルを実行するとリンクが表示されるのでクリックして編集を行ってください。

> ファイルの編集後は**必ず**、メニューの[File]-[Save]を選択してファイルの保存を行ってください。

In [ ]:
%run scripts/edit_conf.py
create_conf(ugroup_name, target_container, target_file)

作成したファイルは、以下のパスに格納されています。

`./edit/{ugroup_name}/{YYYYMMDDHHmmssffffff}/shibboleth2.xml`

`{ugroup_name}` には UnitGroup名が、`{YYYYMMDDHHmmssfffff}` にはファイルを取得したタイムスタンプが入ります。

設定ファイルの配置を行います。次のセルを実行すると以下の手順が実行されます。

1. 編集前と編集後の設定ファイルの差分を表示する
2. 編集した設定ファイルをVCノードに配置する

In [ ]:
%run scripts/edit_conf.py
apply_conf(ugroup_name, target_container, target_file, restart=False)

#### ssl.conf の編集

まず対象となるコンテナを指定します。ここではShibbolethコンテナを対象としているので `shibboleth` を指定します。

In [ ]:
target_container = 'shibboleth'

次に編集対象となる設定ファイルのパスを指定します。指定するのはコンテナ内のパスとなります。

In [ ]:
target_file = '/etc/httpd/conf.d/ssl.conf'

設定ファイル`ssl.conf`をローカル環境に取得しJupyter Notebookの編集機能を用いて編集を行います。次のセルを実行するとリンクが表示されるのでクリックして編集を行ってください。

> ファイルの編集後は**必ず**、メニューの[File]-[Save]を選択してファイルの保存を行ってください。

In [ ]:
%run scripts/edit_conf.py
fetch_conf(ugroup_name, target_container, target_file)

作成したファイルは、以下のパスに格納されています。

`./edit/{ugroup_name}/{YYYYMMDDHHmmssffffff}/shibboleth2.xml`

`{ugroup_name}` には UnitGroup名が、`{YYYYMMDDHHmmssfffff}` にはファイルを取得したタイムスタンプが入ります。

次のセルを実行すると編集の前後での差分を確認することができます。

In [ ]:
show_local_conf_diff(ugroup_name, target_container, target_file)

設定ファイルの配置を行います。次のセルを実行すると以下の手順が実行されます。

1. 編集前と編集後の設定ファイルの差分を表示する
2. 編集した設定ファイル(`ssl.conf`)をVCノードに配置する

In [ ]:
%run scripts/edit_conf.py
apply_conf(ugroup_name, target_container, target_file, restart=False)

### 設定ファイルの変更を反映する

設定ファイルの変更を反映するためにShibbolethコンテナを再起動します。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle/ \
    docker-compose restart shibboleth'

Shibbolethコンテナの状態が `Up`となっていることを確認してください。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle/ docker-compose ps'

## MoodleのShibbolethプラグインの設定

MoodleのShibboleth認証を利用できるようにする設定を行います。

### Shibboleth認証を有効にする

次のセルを実行すると表示されるリンクからMoodleの認証管理画面を開いてください。
> メニューから[サイト管理]--[プラグイン]--[認証]--[認証管理]を選択して同様の画面を開くことができます。

In [ ]:
from IPython.core.display import HTML
HTML(u'<a href="{0}/admin/settings.php?section=manageauths" target="_blank">{1}</a>'.format(
    load_group_var(ugroup_name, 'moodle_url'), u'認証管理'))

以下のような画面が表示されるので Shibboleth の Yes の列をクリックし Shibboleth認証プラグインを有効にしてください。

![認証管理](images/moodle-220-10.png)

### Shibbolethプラグイン設定

Shibbolethプラグインの設定を行います。

次のセルを実行すると表示されるリンクからShibbolethプラグインの設定画面を開いてください。
> メニューから[サイト管理]--[プラグイン]--[認証]--[Shibboleth]を選択して同様の画面を開くことができます。

In [ ]:
HTML(u'<a href="{0}/admin/settings.php?section=authsettingshibboleth" target="_blank">{1}</a>'.format(
    load_group_var(ugroup_name, 'moodle_url'), u'Shibboleth'))

以下のような画面が表示されるので、例えば以下のように項目を設定する。

<dl>
<dt>ユーザ名</dt><dd>HTTP_X_SHIB_EDUPERSONPRINCIPALNAME</dd>
<dt>ShibbolethサービスプロバイダのログアウトハンドラURL</dt><dd>/Shibboleth.sso/Logout</dd>
<dt>データマッピング：名</dt><dd>HTTP_X_SHIB_JAGIVENNAME</dd>
<dt>データマッピング：姓</dt><dd>HTTP_X_SHIB_JASN</dd>
<dt>データマッピング：メールアドレス</dt><dd>HTTP_X_SHIB_MAIL</dd>
<dt>データマッピング：名 - ヨミガナ</dt><dd>HTTP_X_SHIB_GIVENNAME</dd>
<dt>データマッピング：姓 - ヨミガナ</dt><dd>HTTP_X_SHIB_SN</dd>
<dt>データマッピング：所属組織</dt><dd>HTTP_X_SHIB_JAO</dd>
<dt>データマッピング：部署</dt><dd>HTTP_X_SHIB_JAOU</dd>
</dl>

マッピングに指定する値は、基本的に学認の[属性リスト](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=12158166)の名前にプレフィックス`HTTP_X_SHIB_`を付加したものにある。

![Shibboleth設定画面](images/moodle-220-11.png)